In [28]:
import requests
from collections import OrderedDict
from ipy_table import make_table

In [29]:
# From https://docs.google.com/spreadsheets/d/1U79vkTIZ8U0I_N0QtOPZ6ajFkoPH-Ym2yEkdYPPPW74/edit#gid=0
ag_sector_codes = [
'12240', 
'14040', 
'311', 
'31110', 
'31120', 
'31130', 
'31140', 
'31150', 
'31161', 
'31162', 
'31163', 
'31164', 
'31165', 
'31166', 
'31181', 
'31182', 
'31191', 
'31192', 
'31193', 
'31194', 
'31195', 
'312', 
'31210', 
'31220', 
'31261', 
'31281', 
'31282', 
'31291', 
'313', 
'31310', 
'31320', 
'31381', 
'31382', 
'31391', 
'32161', 
'32162', 
'32165', 
'43040', 
'520', 
'52010', 
'72040', 
]

In [30]:
codelist_values = requests.get('http://dashboard.iatistandard.org/stats/current/inverted-publisher/codelist_values.json').json(object_pairs_hook=OrderedDict)

In [31]:
sector_code_lookup = codelist_values['.//sector/@code']
make_table([(sector_code,k,v) for sector_code in ag_sector_codes for k,v in sector_code_lookup[sector_code].items()])

12240,afd,5
12240,amrefnl,1
12240,ausgov,29
12240,be-dgd,3
12240,bmgf,165
12240,bmz,24
12240,cafod,13
12240,cordaid,10
12240,danida,12
12240,dfid,299
12240,ec-devco,100
